In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math
from sort import *
import numpy as np

# 🎥 Input video
cap = cv2.VideoCapture('../weights/cars.mp4')
model = YOLO("../weights/yolov8n.pt")

# ✅ Full HD output setup
output_width = 1920
output_height = 1080
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:  # Fallback if FPS not detected
    fps = 30.0

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('vehicle_counter_output_HD.mp4', fourcc, fps, (output_width, output_height))

# 🧠 Class names (COCO)
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# 🧩 Mask (optional)
mask = cv2.imread("mask.png")
if mask is not None:
    mask = cv2.resize(mask, (int(cap.get(3)), int(cap.get(4))))
else:
    print("⚠️ Mask not found — continuing without mask")

# 🛰️ Tracker setup
tracker = Sort(max_age=20, min_hits=2, iou_threshold=0.3)
totalCount = []

# 🧭 Detection line coordinates
limits = [400, 297, 673, 297]

while True:
    success, img = cap.read()
    if not success:
        break

    # Apply mask if available
    imRegion = cv2.bitwise_and(img, mask) if mask is not None else img

    # 🔰 Overlay logo (top-left corner)
    imgGraphics = cv2.imread('graphics.png', cv2.IMREAD_UNCHANGED)
    if imgGraphics is not None:
        imgGraphics = cv2.resize(imgGraphics, (300, 200))
        if imgGraphics.shape[2] == 3:
            imgGraphics = cv2.cvtColor(imgGraphics, cv2.COLOR_BGR2BGRA)
        img = cvzone.overlayPNG(img, imgGraphics, (20, 20))

    # Run YOLO
    results = model(imRegion, stream=True)
    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if currentClass in ['car', 'truck', 'bus', 'motorbike'] and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # 🧮 Update tracker
    resultsTracker = tracker.update(detections)

    # 🎨 Professional Color Theme
    line_color_default = (0, 115, 255)      # Deep orange-blue for main line
    line_color_counted = (0, 220, 150)      # Emerald green when vehicle crosses
    box_color = (60, 130, 255)              # Royal blue for boxes
    circle_color = (0, 180, 255)            # Light blue for tracking point

    # Draw detection line
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), line_color_default, 5)

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1
        cx, cy = x1 + w // 2, y1 + h // 2

        # Bounding box (clean royal blue)
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=box_color)
        cvzone.putTextRect(img, f'{int(id)}', (max(0, x1), max(35, y1)),
                           scale=1.5, thickness=2, offset=6,
                           colorT=(255, 255, 255), colorR=(40, 40, 40))
        cv2.circle(img, (cx, cy), 5, circle_color, cv2.FILLED)

        # Vehicle counting logic
        if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
            if totalCount.count(id) == 0:
                totalCount.append(id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), line_color_counted, 5)

    # ✅ Save frame to Full HD output video
    frame_hd = cv2.resize(img, (output_width, output_height))
    out.write(frame_hd)

    # Display on screen
    cv2.imshow("Professional Vehicle Counter", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ✅ Release everything
cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Video saved successfully as 'vehicle_counter_output_HD.mp4'")